In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH21-AGENT")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH21-AGENT


# Tools 

- 도구(Tool) : Agent, Chain 또는 LLM이 외부와 상호작용 하기 위한 인터페이스 
- LangChain에서 기본적으로 제공하는 도구, 또는 Custom Tool 구성하여 적용 
- LangChain 도구 리스트 : https://python.langchain.com/v0.1/docs/integrations/tools/


## Built-in Tools 
- LangChain에서 제공하는 사전 정의된 took & toolkit 
    - tool : 단일 도구 
    - toolkit : 여러 도구를 묶어서 하나의 도구로 사용할 수 있음 
    - https://python.langchain.com/docs/integrations/tools/

### Python REPL Tools 
- 파이썬 코드를 REPL 환경에서 실행하기 위한 클래스 제공 
    - REPL : Read-Eval-Print Loop, 파이썬을 바로 실행해 볼 수 있는 대화형 환경 
    - https://python.langchain.com/docs/integrations/tools/python/


In [3]:
# from langchain_experimental.tools import PythonREPLTool

In [4]:
# PythonREPLTool import 오류 해결을 위한 대안 방법들
try:
    from langchain_experimental.tools import PythonREPLTool
    print("PythonREPLTool imported successfully")
except Exception as e:
    print(f"PythonREPLTool import failed: {e}")
    print("Trying alternative import methods...")
    
    # 대안 1: langchain_community에서 import 시도
    try:
        from langchain_community.tools import PythonREPLTool
        print("PythonREPLTool imported from langchain_community")
    except Exception as e2:
        print(f"langchain_community import failed: {e2}")
        
        # 대안 2: 직접 구현
        print("Creating custom PythonREPLTool...")
        from langchain_core.tools import BaseTool
        from typing import Any, Dict
        import subprocess
        import sys
        
        class PythonREPLTool(BaseTool):
            name: str = "python_repl"
            description: str = "A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`."
            
            def _run(self, query: str) -> str:
                """Use the tool."""
                try:
                    # Create a subprocess to run Python code
                    result = subprocess.run(
                        [sys.executable, "-c", query],
                        capture_output=True,
                        text=True,
                        timeout=30
                    )
                    if result.returncode != 0:
                        return f"Error: {result.stderr}"
                    return result.stdout
                except subprocess.TimeoutExpired:
                    return "Error: Code execution timed out"
                except Exception as e:
                    return f"Error: {str(e)}"
        
        print("Custom PythonREPLTool created successfully")

PythonREPLTool imported successfully


In [5]:
python_tool = PythonREPLTool()

print(python_tool.invoke(
    "print (100 + 500)"
))

Python REPL can execute arbitrary code. Use with caution.


600



Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 

#### LLM 적용 

기본 흐름 
1. LLM 모델에게 특정 작업을 수행하는 파이썬 코드 작성 요청 
2. 작성된 코드를 실행하여 결과 획득 
3. 결과 출력

In [6]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [7]:
def print_and_execute(code, debug=True):
    if debug:
        print("CODE: ")
        print(code)
    
    return python_tool.invoke(code)

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)

llm = OpenAI(temperature=0)

chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [9]:
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

CODE: 


import random

# Generate 6 random numbers between 1 and 45
lotto_numbers = random.sample(range(1, 46), 6)

# Sort the numbers in ascending order
lotto_numbers.sort()

# Print the numbers
print(lotto_numbers)
[2, 5, 25, 29, 32, 44]



### 검색API 도구

- Tavily API : 검색 기능을 구현해 놓은 도구 
- [Ref] https://python.langchain.com/docs/integrations/tools/tavily_search/

- 사전준비 : 
    - API 키 발급 : https://app.tavily.com/
    - 발급받은 키를 `.env` 파일에 반영 (TAVILY_API_KEY=tv...)

- 주요 클래스 : 
    - `TavilyAnswer`
    - `TavilySerachResults`
        - API 통해 검색하고 JSON 형식을 결과를 반환 
        - 주요 매개 변수 
            - `max_results` (int): 반환할 최대 검색 결과 수 (기본값: 5)
            - `search_depth` (str): 검색 깊이 ("basic" 또는 "advanced")
            - `include_domains` (List[str]): 검색 결과에 포함할 도메인 목록
            - `exclude_domains` (List[str]): 검색 결과에서 제외할 도메인 목록
            - `include_answer` (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
            - `include_raw_content` (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
            - `include_images` (bool): 쿼리 관련 이미지 목록 포함 여부

In [10]:
from langchain_core.pydantic_v1 import BaseModel
from langchain_community.tools.tavily_search import TavilySearchResults

/Users/yshmbid/Documents/home/github/Agent/agent-env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
# 도구 생성 
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced" # or "basic"
    include_domains=["github.io","wikidocs.net"],
    # exclude_domains=[]
)

/var/folders/5_/cr0lgx655m7fqg_j4q733qbm0000gn/T/ipykernel_27640/1072773976.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(


In [12]:
tool.invoke({
    "query":"LangChain Tool에 대해서 알려주세요"
})

[{'title': 'LangChain, chains and agents, a great piece of engineering ...',
  'url': 'https://datasciencebyexample.github.io/2023/05/27/understanding-langchain-chains-and-agents/',
  'content': '| ``` # Set up a prompt templateclass CustomPromptTemplate(BaseChatPromptTemplate):     # The template to use    template: str     # The list of tools available    tools: List[Tool]         def format_messages(self, **kwargs) -> str:        # Get the intermediate steps (AgentAction, Observation tuples)         # Format them in a particular way        intermediate_steps = kwargs.pop("intermediate_steps")        thoughts = ""        for action, observation in intermediate_steps:            thoughts += action.log            thoughts += f"\\nObservation: {observation}\\nThought: "         # Set the agent_scratchpad variable to that value        kwargs["agent_scratchpad"] = thoughts         # Create a tools variable from the list of tools provided        kwargs["tools"] = "\\n".join([f"{tool.name}:

## Custom Tools 

- LangChain에서 제공하는 빌트인 도구와 별도로, 사용자가 직접 도구를 정의하교 사용 
- 이를 위해 `langchain.tools` 모듈에서 제공하는 `tool` decorator를 사용하여 파이썬 함수를 도구로 변환 

### @tool decorator 
- 파이썬 함수를 도구로 변환 하는 기능 
- 사용 방법 
    - 함수 위에 `@tool` 데코레이터 적용 
    - 필요에 따라 데코레이터 매개변수 설정 

In [13]:
from langchain.tools import tool

In [14]:
# 데코레이터 추가하여 함수 -> 도구로 변환 
# LLM이 함수를 호출할 때 함수의 역할을 확인할 수 있도록 주석을 반드시 추가 (in English)

@tool
def add_numbers(a: int, b: int) -> int:
    """ Add two numbers
    """
    return a + b

@tool
def mult_numbers(a: int, b: int) -> int:
    """ Multiply two numbers
    """
    return a * b

In [15]:
add_numbers.invoke({'a': 10, 'b':5})

15

In [16]:
mult_numbers.invoke({'a': 10, 'b':5})

50

--------
** End of Documents **